In [1]:
import asyncio
import requests
from datetime import datetime, timedelta
import pandas as pd
from scipy.integrate import trapezoid
import math
from statistics import mean
from io import StringIO
import logging

In [2]:
logging.basicConfig(format='%(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',level=logging.DEBUG)

In [3]:
async def send_get_request(url:str='http://participant0.local:5000/',endpoint:str='',type:str='json',key=None,timeout=1):
        """Send GET request to the IP."""
        # get own data
        max_tries = 3

        if key:
            headers = {"Authorization": f"Bearer {key}"}
        else:
            headers = {}
            
        for attempt in range(max_tries):
            try:
                response = requests.get(f"{url}{endpoint}",headers=headers, timeout=timeout)
                response.raise_for_status()
                if type == 'json':
                    res= parse_datetimes(convert_bools(response.json()))
                elif type == 'text':
                    res= response.text
                else:
                    res= response.status_code
                break
            except requests.exceptions.HTTPError as e:
                logging.error(f"HTTP error occurred: {e}")
            except Exception as e:
                logging.error(f'{e}')
                if attempt == max_tries-1: # try up to 3 times
                    return None
                else:
                    logging.debug('SLEEEEEEEEEEEEEEEEEPING')
                    await asyncio.sleep(1+attempt)
        return res

# # convert datetimes to iso formatted strings
# def convert_datetimes(obj):
#     if isinstance(obj, dict):
#         return {k: convert_datetimes(v) for k, v in obj.items()}
#     elif isinstance(obj, list):
#         return [convert_datetimes(i) for i in obj]
#     elif isinstance(obj, datetime):
#         return obj.isoformat()
#     else:
#         return obj

# convert to datetimes from iso formatted strings
def parse_datetimes(obj):
    if isinstance(obj, dict):
        return {k: parse_datetimes(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [parse_datetimes(i) for i in obj]
    elif isinstance(obj, str):
        try:
            return datetime.fromisoformat(obj)
        except ValueError:
            return obj
    else:
        return obj

 # Function to recursively convert "true"/"false" strings to Booleans
def convert_bools(obj):
    if isinstance(obj, dict):
        return {k: convert_bools(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_bools(elem) for elem in obj]
    elif obj == "true":
        return True
    elif obj == "false":
        return False
    else:
        return obj

In [4]:
# get event log to filter out past events
key='patr0QyMjRUmQManT.5731b23d71fa89b29c97d8be90faccf840ad8765bb8d88e66f583ad485f28e77'
baseURL = 'https://api.airtable.com/v0/'
base = 'apptjKq3GAr5CVOQT'
table = 'events'

res = await send_get_request(f'{baseURL}{base}/{table}',key=key)
fields = []
for r in res['records']:
    fields.append(r['fields'])
eventDF = pd.DataFrame(data=fields)

# convert to date
eventDF['date'] = pd.to_datetime(eventDF['date'])

# drop unnecessary columns
eventDF = eventDF.drop(columns=['modified','notes','network'])
pastEventsDF=eventDF[eventDF['date']<datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)]
pastEventDates = [d.date() for d in list(pastEventsDF['date'])]
pastEventDates

DEBUG    [connectionpool.py:1049] Starting new HTTPS connection (1): api.airtable.com:443
DEBUG    [connectionpool.py:544] https://api.airtable.com:443 "GET /v0/apptjKq3GAr5CVOQT/events HTTP/1.1" 200 436


[datetime.date(2025, 7, 28), datetime.date(2025, 8, 2)]

In [5]:
async def getPastData():
    # get file list
    fileList = await send_get_request('http://participant0.local:5000/api/files?source=plugs',type='json')
    logging.debug(f'all files: {fileList}')

    filteredFileList = []
    for f in fileList:
        dStr = f.split('_')[1].replace('.csv','')
        dDt = datetime.strptime(dStr, "%Y-%m-%d")
        # get only dates within the last 30 days
        if datetime.now() - dDt <= timedelta(days=30):            
            filteredFileList.append(dDt)
    logging.debug(f'filtered file list: {filteredFileList}')

    data = []
    for f in filteredFileList:
        #filter out today's data
        fToGet = f.strftime("%Y-%m-%d")
        if datetime.now().date().strftime("%Y-%m-%d")  not in fToGet:    
            r = await send_get_request(f'http://participant0.local:5000/api/data?source=plugs&date={fToGet}',type='text')
            if type(r) == tuple: # the tuple includes the response code, which we don't care about
                r = r[0]
            data.append(r)

    #parse response
    parsedData = []
    for d in data:
        tempDF = pd.read_csv(StringIO(d), na_values=["", "NA", "NaN", "null", "NULL"])
        tempDF['datetime'] = pd.to_datetime(tempDF['datetime'])
        cleanedTempDF = tempDF.dropna()
        parsedData.append(cleanedTempDF)

    return parsedData

# get event windows
def getEventWindows(data,eTime = 14):
    eW = []
    for d in data:
        #get on timestamps between event start and end times
        eW.append(d[[(d > d.replace(hour=eTime,minute=0,second=0,microsecond=0)) and (d <= d.replace(hour=eTime+4,minute=0,second=0,microsecond=0)) for d in d['datetime']]])
    
    return eW


In [6]:
#get all past data for average seed value
data = await getPastData()

DEBUG    [connectionpool.py:241] Starting new HTTP connection (1): participant0.local:5000
DEBUG    [connectionpool.py:544] http://participant0.local:5000 "GET /api/files?source=plugs HTTP/1.1" 200 278
DEBUG    [1552156714.py:4] all files: ['plugs_2025-07-22.csv', 'plugs_2025-07-24.csv', 'plugs_2025-07-25.csv', 'plugs_2025-07-26.csv', 'plugs_2025-07-27.csv', 'plugs_2025-07-28.csv', 'plugs_2025-07-29.csv', 'plugs_2025-07-30.csv', 'plugs_2025-07-31.csv', 'plugs_2025-08-01.csv', 'plugs_2025-08-02.csv', 'plugs_2025-08-03.csv']
DEBUG    [1552156714.py:13] filtered file list: [datetime.datetime(2025, 7, 22, 0, 0), datetime.datetime(2025, 7, 24, 0, 0), datetime.datetime(2025, 7, 25, 0, 0), datetime.datetime(2025, 7, 26, 0, 0), datetime.datetime(2025, 7, 27, 0, 0), datetime.datetime(2025, 7, 28, 0, 0), datetime.datetime(2025, 7, 29, 0, 0), datetime.datetime(2025, 7, 30, 0, 0), datetime.datetime(2025, 7, 31, 0, 0), datetime.datetime(2025, 8, 1, 0, 0), datetime.datetime(2025, 8, 2, 0, 0), dateti

In [7]:
# get event windows
def listToDict(data):
    rD = {}
    for d in data:
        if len(d)>0:
            rD[d['datetime'].iloc[0].date().strftime("%Y-%m-%d")]=d
    
    return rD

# buckets df with datetime within an event window into hourly buckets
# args: a dataframe with datetimes
def hourlyBuckets(tempDF, tempStartTime:float, eventDuration:float=4) -> list[pd.DataFrame]:
    hourlyPower = []
    for h in range(eventDuration):
        #print(tempDF['datetime'])
        ts = tempStartTime + timedelta(hours=h)
        te = tempStartTime + timedelta(hours=h + 1)
        filteredTempDF = (tempDF[(tempDF['datetime']> ts) & (tempDF['datetime']<= te)]).copy() #data within the hour
        #filteredTempDF = increments(filteredTempDF,ts)
        hourlyPower.append(filteredTempDF)
    return hourlyPower


#args: a dataframe with datetime column
# returns df with added increments column based on an hour
def increments(df,fm=0)->pd.DataFrame:
    if fm==0:
        firstMeasurement = df['datetime'].min()
    else:
        firstMeasurement = fm

    #print(firstMeasurement)
    incList = []
    for r in range(len(df['datetime'])):
        incSec = (df['datetime'].iloc[r] - firstMeasurement).total_seconds()/60/60 #must convert back from seconds
        incList.append(incSec)
    df['increments'] = incList
    return df

#args: power and time increments (relative to the hour) for a given hour
# returns the energy (Wh) for the hour
def getWh(p:list[float],t:list[datetime])->float:
    e = trapezoid(y=p, x=t)
    return e

In [8]:
windowData = getEventWindows(data)
#filter all data to event windows
windowDict= listToDict(windowData)
#windowDict

# remove past events and weekends

In [10]:
windowDictBuckets = {}
eTime = 14
for k,v in windowDict.items():
    # create hourly buckets for each day
    formattedStartTime = v['datetime'].iloc[0].replace(hour=eTime,minute=0,second=0,microsecond=0)
    hourly = hourlyBuckets(v,formattedStartTime)

    # add increments within each hour
    incs = []
    for i,h in enumerate(hourly):
        # the increments function adds a column for the increment of a specific datapoint
        incs.append(increments(h,formattedStartTime+timedelta(hours=i)))

    hourlyEnergy = []
    for inc in incs:
        hourlyEnergy.append(getWh(inc['ac-W'],inc['increments']))
        if (math.isnan(hourlyEnergy[-1])):
            hourlyEnergy[-1] = 0.0

    windowDictBuckets[k]=hourlyEnergy

#windowDictBuckets

In [11]:
peakLoad = 0
for k,v in windowDictBuckets.items():
    for i,h in enumerate(v):
        peakLoad = max(peakLoad,h)

peakLoad

np.float64(410.51729666666665)

In [12]:
#pastEventDates

In [13]:
# t = list(windowDictBuckets.keys())[0]
# datetime.strptime(t, "%Y-%m-%d")

In [14]:
pastEventDates

[datetime.date(2025, 7, 28), datetime.date(2025, 8, 2)]

In [15]:
pastEventPriorDates = [d - timedelta(days=1) for d in pastEventDates]
pastEventPriorDates

[datetime.date(2025, 7, 27), datetime.date(2025, 8, 1)]

In [16]:
filteredBuckets = {}
holidays =  ['2025-09-01']

for k,v in windowDictBuckets.items():
    kDT = datetime.strptime(k, "%Y-%m-%d")
    #drop holidays
    if not (k in holidays):
        #drop DR event days
        if not (kDT.date() in pastEventDates):
            #drop DR event prior days
            if not (kDT.date() in pastEventPriorDates):
            # drop weekends
                if kDT.weekday() <=4: # filter out weekends
                    filteredBuckets[k]=v

filteredBuckets

{'2025-07-22': [np.float64(327.08983430555554),
  np.float64(56.76919861111111),
  np.float64(56.718149861111094),
  np.float64(205.02579972222225)],
 '2025-07-24': [np.float64(383.16116458333335),
  np.float64(347.00701847222217),
  np.float64(380.77539847222226),
  np.float64(378.90866180555554)],
 '2025-07-25': [np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0)]}

In [17]:
# checkD = datetime.now().date() - timedelta(days=1)
# checkD.strftime("%Y-%m-%d")

In [18]:
# list(filteredBuckets.keys())

In [19]:
# count = 0

def tenDayCBL(b,p):
    cblWindowAvg = {}
    avgList = []

    # loop backwards through dates
    for d in range(1,31):
        checkD = datetime.now().date() - timedelta(days=d)
        checkDstr = checkD.strftime("%Y-%m-%d")

        if checkDstr in list(b.keys()):
            if mean(b[checkDstr])> p * .25:
                if len(cblWindowAvg.keys()) == 0:
                    p = mean(b[checkDstr])
                else:
                    avgList.append(mean(b[checkDstr]))
                    p = mean(avgList)
                cblWindowAvg[checkDstr]=mean(b[checkDstr])

        if len(cblWindowAvg.keys()) >= 10:
            break

    # if not enough days, repeat without removing low days
    if len(cblWindowAvg.keys()) < 10:
        cblWindowAvg = {}
        avgList = []
        # loop backwards through dates
        for d in range(1,31):
            checkD = datetime.now().date() - timedelta(days=d)
            checkDstr = checkD.strftime("%Y-%m-%d")
            if checkDstr in list(b.keys()):
                cblWindowAvg[checkDstr]=mean(b[checkDstr])
            if len(cblWindowAvg.keys()) >= 10:
                break
                
    return cblWindowAvg

tD = tenDayCBL(filteredBuckets,peakLoad)

#get highest 5
def fiveDayCBL(t):
    allV = list(t.values())
    allV.sort(reverse=True)

    f = {}
    for v in allV:
        for k,vt in t.items():
            if v == vt:
                f[k]=v
    return f

CBLbasis = fiveDayCBL(tD)

# hourlyAvg = [h / len(windowDictBuckets.keys()) for h in hourlySums]
#  = hourlyAvg

In [20]:
def avgCBL(dates,buckets):
    hourlyLists = [[],[],[],[]]
    for d in dates:
        hourlyLists[0].append(buckets[d][0])
        hourlyLists[1].append(buckets[d][1])
        hourlyLists[2].append(buckets[d][2])
        hourlyLists[3].append(buckets[d][3])
    hourlyAvg = []
    for h in hourlyLists:
        hourlyAvg.append(mean(h))

    return hourlyAvg
        
avgCBL(list(CBLbasis.keys()), filteredBuckets)

[np.float64(236.75033296296297),
 np.float64(134.5920723611111),
 np.float64(145.8311827777778),
 np.float64(194.64482050925926)]

In [21]:
datetime.now().time().hour

2

In [22]:
stateDict={"csrp":{"baselineW":0,"baselineTS":False,"now":False,"upcoming":False,"avgPerf":100},
                    "dlrp":{"baselineW":0,"now":False,"upcoming":False,"avgPerf":100},
                    "datetime":datetime.now(),
                    "eventPause":{"datetime":datetime.now(), "state":False},
                    "relays":{'bat-in':True,'bat-out':True,'ac':True}}

In [28]:
if not 'baselineTS' in stateDict.keys():
    print('y')

y


In [31]:
l = [4,4,3]
for b,i in enumerate(l):
    print(b)
    print(i)

0
4
1
4
2
3


In [34]:
(10-1)/10

0.9